In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import pandas_plink as pk
import numpy as np
import os
import matplotlib.pyplot as plt
import subprocess as sp
import h5py
import statsmodels.formula.api as smf
import statsmodels.api as sm
import seaborn as sns

In [ ]:
# input files
ROOTDIR = '/ess/p33/cluster/users/espehage/geno_nn/src/ukb'
OUTDIR = os.path.join(ROOTDIR, 'output')
UKBLAKE = '/ess/p33/data/durable/s3-api/ukblake'
UKBDATA = '/ess/p33/data/durable/s3-api/ukbdata'
geno = os.path.join(UKBLAKE, 'genetics')
pheno_ukb = os.path.join(UKBLAKE, 'phenotypes')
iidsets_out = os.path.join(ROOTDIR, 'iidsets')
pheno_out = os.path.join(ROOTDIR, 'pheno')

SUMSTATS = '/cluster/projects/p33/groups/biostat/SUMSTATv3/v3.1/STD_GRCh37/GIANT_HEIGHT_2022.sumstats.gz'
TRAIT = 'height'
COVAR = '/ess/p33/cluster/users/espehage/gwas_ukb_linn_loneliness/src/pheno/processed/COVAR.DEFAULT.tsv.gz'

GENO_PREFIX = 'ukb_cal_merged'


In [ ]:
# extract the top SNPs ranked by -log10(P-value)
# for subsequent readout from PLINK .bed files
sumstats = pd.read_csv(SUMSTATS, sep='\s+')

In [ ]:
sumstats.head()

In [ ]:
# !mkdir -p data
# os.makedirs(os.path.join(OUTDIR, ))

In [ ]:
# extract the quasi-independent SNP IDs from Yengo et al. 2022
quasi_indep = pd.read_excel(os.path.join(ROOTDIR, '41586_2022_5275_MOESM6_ESM.xlsx'), usecols=['Chromosome', 'Position (hg37)'])
quasi_indep.rename(columns={'Chromosome': 'CHR', 'Position (hg37)': 'POS'}, inplace=True)

sumstats = sumstats.merge(quasi_indep, on=['CHR', 'POS'])

# store SNPs for extraction
snps_p = sumstats[['RSID', 'P']].copy()
top_snps = f'{OUTDIR}/{TRAIT}.top_indep_snps'
snps_p['RSID'].to_csv(top_snps, sep='\t', index=False, header=False)

In [ ]:
# dump top SNPs to file
sumstats.to_csv(os.path.join(OUTDIR, 'GIANT_HEIGHT_2022.top_indep_snps.sumstats.gz'), sep='\t', index=False)

In [ ]:
pd.read_csv(top_snps, header=None)

In [ ]:
# produce PLINK bed file with subset of SNPs
bed_output = f'{OUTDIR}/{GENO_PREFIX}.top_indep_snps'
call = f'plink --bfile {os.path.join(ROOTDIR, "cal", GENO_PREFIX)} --memory 64000 --threads 16 --extract {top_snps} --make-bed --out {bed_output}'
proc = sp.run(call.split(), capture_output=True)
proc

In [ ]:
G = pk.read_plink1_bin(bed_output + '.bed')
G

In [ ]:
# get phenotype value of interest for included individuals
covar = pd.read_csv(COVAR, sep='\t')

# phenotypic info
pheno = pd.read_csv(os.path.join(OUTDIR, 'WHITE_BRITISH_SELF_GEN_noREL_noEXL_pheno.tsv'), sep='\t')

# merge
covar = covar.merge(pheno, on='IID')

# match IIDs with genotyped samples
samples = pd.DataFrame({'IID': np.array(G.sample).astype(int)})

# merge
covar = pd.merge(covar, samples, on='IID', how='inner')
covar

In [ ]:
def standardize_resid(df, x, cont_variables=[], cat_variables=[]):
    formula = f'{x}~'
    if len(cont_variables) > 0:
        formula += '+'.join(cont_variables)
        if len(cat_variables) > 0:
            formula += '+'
    if len(cat_variables) > 0:
        formula += '+'.join([f'C({var})' for var in cat_variables])
    print(formula)
    model = smf.ols(formula, data=sm.add_constant(df)).fit()
    return model.get_influence().resid_studentized_internal

In [ ]:
# prediction value, i.e., height regularized on SEX, AGE, PC1-20 variables
covar['height_resid'] = np.NaN
covar.loc[~covar.height.isna(), 'height_resid'] = standardize_resid(covar, 'height', ['AGE'] + [f'PC{i}' for i in range(1, 21)], ['SEX'])
covar

In [ ]:
sns.jointplot(data=covar, x='height', y='height_resid', hue='SEX', marker=',', s=2)

In [ ]:
# write to file
with h5py.File(os.path.join(OUTDIR, 'ukb_sample.h5'), 'w') as f:
    f.create_dataset('x_data', data=G.sel(sample=list(covar['IID'].astype(str))).compute().astype('uint8'))
    f.create_dataset('y_data', data=np.array(covar['height_resid']))
# append dataframe
covar.to_hdf(os.path.join(OUTDIR, 'ukb_sample.h5'), mode='a', key='df')

In [ ]:
with h5py.File(os.path.join(OUTDIR, 'ukb_sample.h5'), 'r') as f:
    print(list(f.items()))